一、标准化（Z-Score），或者去除均值和方差缩放

需要进行标准化的模型：带梯度下降（梯度更新）的，如线性回归，距离计算的，如SVM等

不需要的模型：树类：决策树，RF,不涉及距离计算

公式为：(X-mean)/std  计算时对每个属性/每列分别进行。将数据按期属性（按列进行）减去其均值，并处以其方差。得到的结果是，对于每个属性/每列来说所有数据都聚集在0附近，方差为1。

Z-score标准化就是将数据变化为服从均值为0，方差为1的正太分布，对于不知道数据最大最小值的时候适用，但是会改变数据的分布。

实现时，有两种不同的方式：

使用sklearn.preprocessing.scale()函数，可以直接将给定数据进行标准化。



In [ ]:
sklearn.preprocessing.scale(X, axis=0, with_mean=True, with_std=True, copy=True)

使用sklearn.preprocessing.StandardScaler类，使用该类的好处在于可以保存训练集中的参数（均值、方差）直接使用其对象转换测试集数据。
类可以进行transform.
一般会把train和test集放在一起做标准化，或者在train集上做标准化后，用同样的标准化器去标准化test集，此时可以用scaler

In [61]:
import numpy as np
import pandas as pd
df = pd.read_csv("forestfires.csv")

In [18]:
df.head()

,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
0,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0
1,7,4,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0
2,7,4,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0
3,8,6,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0
4,8,6,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0


In [17]:
df1 = df.loc[:,"FFMC":"rain"]
df1.head()

,FFMC,DMC,DC,ISI,temp,RH,wind,rain
0,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0
1,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0
2,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0
3,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2
4,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0


In [21]:
from sklearn.cross_validation import train_test_split
# help(train_test_split)
X_train, X_test, y_train, y_test = train_test_split(df1,df["area"], test_size=0.33)

In [25]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()

In [26]:
ss.fit(X_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [28]:
X_train_ss = ss.transform(X_train)
X_test_ss = ss.transform(X_test)

In [29]:
X_train_ss

array([[ 0.17149363,  2.0184335 ,  0.80905435, ...,  0.80378477,
        -0.80001322, -0.12283327],
       [ 0.36612027,  0.05630901,  0.47253065, ..., -0.6651036 ,
         1.04139059, -0.12283327],
       [-0.14477466,  2.64390766,  1.2071564 , ..., -0.05306678,
        -0.56241272, -0.12283327],
       ..., 
       [-0.19343132, -0.09254181, -0.06951567, ...,  1.29341423,
        -0.80001322, -0.12283327],
       [ 0.3904486 , -0.86837034, -0.44800678, ..., -0.35908519,
         0.8037901 , -0.12283327],
       [-0.16910299, -0.21883948,  0.32819415, ..., -0.6651036 ,
        -1.09701383, -0.12283327]])

In [31]:
X_train_ss.mean(axis = 0)

array([ -1.32970064e-15,   1.12947545e-16,  -2.20761110e-16,
        -5.13397930e-18,  -1.18081524e-16,  -7.18757103e-17,
        -3.64512531e-16,  -7.70096896e-18])

In [33]:
X_train_ss.std(axis = 0)

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.])

二、将属性缩放到一个指定范围

除了上述介绍的方法之外，另一种常用的方法是将属性缩放到一个指定的最大和最小值（通常是1-0）之间，这可以通过preprocessing.MinMaxScaler类实现。

使用这种方法的目的包括：

1、对于方差非常小的属性可以增强其稳定性。

2、维持稀疏矩阵中为0的条目。

MinMax标准化适用于知道最大值和最小值的数据，标准化后的数据将会落在[0,1]区间内，如果有异常值则会将数据分布不均匀，向异常值的另一侧堆积。


In [ ]:
sklearn.preprocessing.MinMaxScaler(feature_range=(0, 1), copy=True)


三、正则化（Normalization）
正则化的过程是将每个样本缩放到单位范数（每个样本的范数为1），如果后面要使用如二次型（点积）或者其它核方法计算两个样本之间的相似性这个方法会很有用。计算样本与样本距离时使用。

Normalization主要思想是对每个样本计算其p-范数，然后对该样本中每个元素除以该范数，这样处理的结果是使得每个处理后样本的p-范数（l1-norm,l2-norm）等于1。

p-范数的计算公式：||X||p=(|x1|^p+|x2|^p+...+|xn|^p)^1/p

该方法主要应用于文本分类和聚类中。例如，对于两个TF-IDF向量的l2-norm进行点积，就可以得到这两个向量的余弦相似性。

正则化数据和之前的标准化数据不同，之前标准化数据是针对特征来说的，而现在正则化是对样本来做的，是用样本数据除以他的范式



In [36]:
from sklearn.preprocessing import Normalizer
normalizer = Normalizer()#l2正则化
normalizer.fit(X_train)#这个fit函数没有任何作用
X_train_ss = normalizer.transform(X_train)

In [37]:
X_train_ss

array([[ 0.11430916,  0.30998247,  0.9406795 , ...,  0.07237916,
         0.00336937,  0.        ],
       [ 0.13472743,  0.1719087 ,  0.97400348, ...,  0.04957503,
         0.00845692,  0.        ],
       [ 0.09934549,  0.31904975,  0.94097673, ...,  0.04840755,
         0.00341053,  0.        ],
       ..., 
       [ 0.16309043,  0.19549131,  0.95899347, ...,  0.11946691,
         0.00488728,  0.        ],
       [ 0.20600115,  0.12560502,  0.96497619, ...,  0.08685454,
         0.01202601,  0.        ],
       [ 0.1394921 ,  0.15402897,  0.97613541, ...,  0.05258017,
         0.00340225,  0.        ]])

阈值划分

连续特征值可以根据阈值划分进行二值化，大于阈值的值为1，否则为0,将连续变量划分为离散值

使用LR时，将连续值大量转换为0,1，这样可以变成稀疏矩阵，加快计算速度

In [40]:
df['DC'].mean()

547.9400386847191

In [41]:
from sklearn import preprocessing
scaler = preprocessing.Binarizer(threshold=547)#threshold为阈值
x_scaled = scaler.fit_transform(df['DC'])


d:\anaconda2\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
d:\anaconda2\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


In [45]:
x_scaled.shape

(1L, 517L)

In [47]:
df['DC_bi'] = x_scaled[0,:]

In [48]:
df.head()

,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,DC_bi
0,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0,0.0
1,7,4,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0,1.0
2,7,4,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0,1.0
3,8,6,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0,0.0
4,8,6,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0,0.0


如果cut传入的是数字n，那么就会均分成n份。可以用这个函数替代。

In [ ]:
pd.cut(df['DC'],5)

离散变量编码

例如性别有‘男’， ‘女’，然而计算机的许多模型都只能在数值型数据当中进行计算，如果我们简单的将‘男’为1，‘女’为0，虽然也可以完成转换，但是在转换的过程当中我们引入了大小关系，就是‘女’ < ‘男’，这会对后续模型应用造成不必要的困扰。 
解决方法为OneHotEncode，就是将其转化为二进制串，除了当前值所在位置为1，其他全部为0，如[0,0,1,0,0], [0,1,0,0,0].。性别可表示为男为[1,0]，女为[0,1]，这样一个性别特征就转化成了两个特征。

In [ ]:
sklearn.preprocessing.OneHotEncoder(n_values='auto', \
categorical_features='all', dtype=<type 'numpy.float64'>, sparse=True, handle_unknown='error')`

通常使用pandas 的 get_dummies 替代之进行one_hot 编码

In [52]:
pd.get_dummies(data = df,columns = ['month','day'])

,X,Y,FFMC,DMC,DC,ISI,temp,RH,wind,rain,...,month_nov,month_oct,month_sep,day_fri,day_mon,day_sat,day_sun,day_thu,day_tue,day_wed
0,7,5,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,7,4,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,7,4,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,8,6,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,8,6,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
5,8,6,92.3,85.3,488.0,14.7,22.2,29,5.4,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
6,8,6,92.3,88.9,495.6,8.5,24.1,27,3.1,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
7,8,6,91.5,145.4,608.2,10.7,8.0,86,2.2,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
8,8,6,91.0,129.5,692.6,7.0,13.1,63,5.4,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
9,7,5,92.5,88.0,698.6,7.1,22.8,40,4.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [77]:
df.DC_nan = df.DC

In [78]:
df.loc[df['DC']>600,'DC_nan'] = np.nan

In [79]:
df.head()

,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,DC_nan
0,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0,94.3
1,7,4,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0,NaN
2,7,4,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0,NaN
3,8,6,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0,77.5
4,8,6,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0,102.2


缺失值处理

由于数据收集的过程，部分样本的属性值有缺失，在处理这部分值的时候可以人工根据经验补充，或者使用均值，出现频率较高的值，中值补充。

In [80]:
from sklearn.preprocessing import Imputer
imp = Imputer(missing_values=np.nan, strategy='mean', axis=0)
#strategy = 'mean', 'median', 'most_frequent'
x = imp.fit_transform(df.DC_nan)

d:\anaconda2\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
d:\anaconda2\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


In [82]:
x.shape

(1L, 183L)

In [83]:
df.fillna(-999)

,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,DC_nan
0,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.00,94.3
1,7,4,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.00,-999.0
2,7,4,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.00,-999.0
3,8,6,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.00,77.5
4,8,6,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.00,102.2
5,8,6,aug,sun,92.3,85.3,488.0,14.7,22.2,29,5.4,0.0,0.00,488.0
6,8,6,aug,mon,92.3,88.9,495.6,8.5,24.1,27,3.1,0.0,0.00,495.6
7,8,6,aug,mon,91.5,145.4,608.2,10.7,8.0,86,2.2,0.0,0.00,-999.0
8,8,6,sep,tue,91.0,129.5,692.6,7.0,13.1,63,5.4,0.0,0.00,-999.0
9,7,5,sep,sat,92.5,88.0,698.6,7.1,22.8,40,4.0,0.0,0.00,-999.0
